<a href="https://colab.research.google.com/github/HWKKK/AIFFEL_quest_cr/blob/main/MainQuest/Quest02/MainQuest02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#가랏 몬스터볼! 전설의 포켓몬 찾아 삼만리



## Step 0. 목차

1. 데이터 살펴보기   
  * kaggle 사이트와 기존 지식을 아울러, feature들의 의미를 확인한다.
  * 각 feature의 특징 혹은 관계를 파악하여 중요한 feature를 찾는다.
  * 결측치 존재 여부를 확인한다.

2. 데이터 전처리
  * 데이터 살펴보기에서 확인한 중요한 feature에 대해 전처리를 수행한다.
  * 그 외 문자형 데이터에 대해 숫자형 인코딩을 수행한다.

3. 기본 모델 학습
  * 전처리하지 않은 데이터로 기본 모델을 학습한다

4. 베이스라인 모델 학습 및 분석
  * Decision Tree 모델에 대한 학습을 진행하고 분석한다.

5. 비교 모델 학습 및 분석
  * 베이스라인 모델에 대한 분석 결과를 바탕으로 추가적으로 시행해 볼 모델을 선정하며 학습&분석 한다.
  * 추가적인 성능 향상 시도를 진행한다.

6. 결론 및 회고



---

##Step 1. 데이터 살펴보기

###라이브러리 가져오기

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


---



###데이터셋 가져오기 및 생성

In [ ]:
import os
from google.colab import files

'''
# 프로젝트 루트 디렉토리 기준으로 파일 경로 설정
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))  # 프로젝트 루트 폴더
DATA_PATH = os.path.join(BASE_DIR, 'data', 'Pokemon.csv')  # 데이터 파일 경로

# CSV 파일 읽기
original_data = pd.read_csv(DATA_PATH)
'''

# 파일 직접 업로드
uploaded = files.upload()

# 파일 이름 확인
csv_filename = list(uploaded.keys())[0]  # 업로드된 파일 이름 가져오기
print(f"파일명: {csv_filename}")

original_data = pd.read_csv(csv_filename)
pokemon = original_data.copy() #원본 데이터는 그대로 보존하며, 실제로 사용할 원본 데이터의 카피본 생성
legendary = pokemon[pokemon["Legendary"] == True].reset_index(drop=True) #전설의 포켓몬 데이터셋 생성
ordinary = pokemon[pokemon["Legendary"] == False].reset_index(drop=True) #일반 포켓몬 데이터셋 생성


Saving Pokemon.csv to Pokemon (2).csv
파일명: Pokemon (2).csv




---



###Data Description

kaggle 페이지에서 확인할 수 있는 데이터 정보

####Basic Description
This data set includes 721 Pokemon, including their number, name, first and second type, and basic stats

####Data fields
- #: ID for each pokemon
- Name: Name of each pokemon
 -Type 1: Each pokemon has a type, this determines weakness/resistance to attacks
- Type 2: Some pokemon are dual type and have 2
- Total: sum of all stats that come after this, a general guide to how strong a pokemon is
- HP: hit points, or health, defines how much damage a pokemon can withstand before fainting
- Attack: the base modifier for normal attacks (eg. Scratch, Punch)
- Defense: the base damage resistance against normal attacks
- SP Atk: special attack, the base modifier for special attacks (e.g. fire blast, bubble beam)
- SP Def: the base damage resistance against special attacks
- Speed: determines which pokemon attacks first each round



---

####결측치 확인

type 2 에 결측치가 있다.   
하지만, 원래 모든 포켓몬은 1개 또는 2개의 type을 가진다.   
따라서 type 2의 결측치는 type을 한 개만 가지고 있는 포켓몬으로 보인다.

이외의 feature에서 결측치 확인되지 않는 것으로 보아, 이상 없는 것으로 보임

In [ ]:
pokemon.isnull().sum()

,0
#,0
Name,0
Type 1,0
Type 2,386
Total,0
HP,0
Attack,0
Defense,0
Sp. Atk,0
Sp. Def,0




---



###Feature 확인

각 feature 별 기본 특성을 정리하면 아래와 같다.   
** Description에서 언급된 내용은 제외


- Name: 중복되는 이름은 없음. 단, 일부 특수 케이스에 대해 부분 중복이 있을 수 있음.
  * (e.g) 한 포켓몬의 진화 형태가 두가지일 경우,   
      * Charizard X
      * Charizard Y
- Type: 한 포켓몬이 1개 또는 2개의 타입을 가짐. 또한, 타입은 18 종류로 구성되어 있음.
- Total: 일반적으로 전설의 포켓몬은 600을 초과하며, 일반 포켓몬은 600 이하이다.   
(단, 특수 진화한 일반포켓몬이나 일부 전설의 포켓몬은 600을 초과하거나 밑돌기도 한다.)
- 세부 스탯: 포켓몬의 특성에 따라 특정 스탯이 타 스탯 대비 높은 경우가 존재한다. 하지만 이것은 개개인의 특성일 뿐, 전설의 포켓몬과 일반 포켓몬을 구분하는 기준으로는 적절하지 않다.

In [ ]:
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


In [ ]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


#####이름의 길이에 따른 전설의 포켓몬일 확률

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#모든 포켓몬의 이름 길이를 "name_count"에 저장한다.
#pokemon["name_count"] = pokemon["Name"].apply(lambda i: len(i))
legendary["name_count"] = legendary["Name"].apply(lambda i: len(i))
ordinary["name_count"] = ordinary["Name"].apply(lambda i: len(i))

print('전설 포켓몬의 이름 길이가 10 이상일 확률: ', round(len(legendary[legendary["name_count"] > 9]) / len(legendary) * 100, 2), "%")
print('일반 포켓몬의 이름 길이가 10 이상일 확률: ', round(len(ordinary[ordinary["name_count"] > 9]) / len(ordinary) * 100, 2), "%")

전설 포켓몬의 이름 길이가 10 이상일 확률:  41.54 %
일반 포켓몬의 이름 길이가 10 이상일 확률:  15.65 %


이름에 특정 토큰이 있는지 여부에 따른 전설의 포켓몬일 확률   

포켓몬의 이름은 네가지 타입이 있음.
- 한 단어면 `ex. Venusaur`
- 두 단어이고, 앞 단어는 두 개의 대문자를 가지며 대문자를 기준으로 두 부분으로 나뉘는 경우 `ex. VenusaurMega Venusaur`
- 이름은 두 단어이고, 맨 뒤에 X, Y로 성별을 표시하는 경우 `ex. CharizardMega Charizard X`
- 알파벳이 아닌 문자를 포함하는 경우 `ex. Zygarde50% Forme`

In [ ]:
#이름에 알파벳이 아닌 문자가 들어간 경우 전처리
pokemon["Name_nospace"] = pokemon["Name"].apply(lambda i: i.replace(" ", ""))
pokemon["name_isalpha"] = pokemon["Name_nospace"].apply(lambda i: i.isalpha())

#남아있는 알파벳이 아닌 것을 포함하는 경우 확인
pokemon[pokemon["name_isalpha"] == False]

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Name_nospace,name_isalpha
34,29,Nidoran♀,Poison,NaN,275,55,47,52,40,40,41,1,False,Nidoran♀,False
37,32,Nidoran♂,Poison,NaN,273,46,57,40,40,40,50,1,False,Nidoran♂,False
90,83,Farfetch'd,Normal,Flying,352,52,65,55,58,62,60,1,False,Farfetch'd,False
131,122,Mr. Mime,Psychic,Fairy,460,40,45,65,100,120,90,1,False,Mr.Mime,False
252,233,Porygon2,Normal,NaN,515,85,80,90,105,95,60,2,False,Porygon2,False
270,250,Ho-oh,Fire,Flying,680,106,130,90,110,154,90,2,True,Ho-oh,False
487,439,Mime Jr.,Psychic,Fairy,310,20,25,45,70,90,60,4,False,MimeJr.,False
525,474,Porygon-Z,Normal,NaN,535,85,80,70,135,75,90,4,False,Porygon-Z,False
794,718,Zygarde50% Forme,Dragon,Ground,600,108,100,121,81,95,95,6,True,Zygarde50%Forme,False


In [ ]:
#9마리밖에 안되므로 손수 replace
pokemon = pokemon.replace(to_replace="Nidoran♀", value="Nidoran male")
pokemon = pokemon.replace(to_replace="Nidoran♂", value="Nidoran female")
pokemon = pokemon.replace(to_replace="Farfetch'd", value="Farfetchd")
pokemon = pokemon.replace(to_replace="Mr. Mime", value="Mr Mime")
pokemon = pokemon.replace(to_replace="Porygon2", value="Porygon Two")
pokemon = pokemon.replace(to_replace="Ho-oh", value="Ho Oh")
pokemon = pokemon.replace(to_replace="Mime Jr.", value="Mime Jr")
pokemon = pokemon.replace(to_replace="Porygon-Z", value="Porygon Z")
pokemon = pokemon.replace(to_replace="Zygarde50% Forme", value="Zygarde Forme")

pokemon.loc[[34, 37, 90, 131, 252, 270, 487, 525, 794]]


# 바꿔준 'Name' 컬럼으로 'Name_nospace'를 만들기
pokemon["Name_nospace"] = pokemon["Name"].apply(lambda i: i.replace(" ", ""))
pokemon["name_isalpha"] = pokemon["Name_nospace"].apply(lambda i: i.isalpha())

이제 이름 안에 알파벳 밖에 없으므로,   
이름을 `대문자로 시작해서 소문자로 끝나는 토큰`으로 분리


In [ ]:
import re

#토큰화 함수
def tokenize(name):
    tokens = []
    name_split = name.split(" ")

    for part_name in name_split:
        a = re.findall('[A-Z][a-z]*', part_name)
        tokens.extend(a)

    return np.array(tokens)

#전설의 포켓몬의 이름에 사용된 토큰 모으기
all_tokens = list(legendary["Name"].apply(tokenize).values)

token_set = []
for token in all_tokens:
    token_set.extend(token)

print(len(set(token_set)))
print(token_set)

65
['Articuno', 'Zapdos', 'Moltres', 'Mewtwo', 'Mewtwo', 'Mega', 'Mewtwo', 'X', 'Mewtwo', 'Mega', 'Mewtwo', 'Y', 'Raikou', 'Entei', 'Suicune', 'Lugia', 'Ho', 'Regirock', 'Regice', 'Registeel', 'Latias', 'Latias', 'Mega', 'Latias', 'Latios', 'Latios', 'Mega', 'Latios', 'Kyogre', 'Kyogre', 'Primal', 'Kyogre', 'Groudon', 'Groudon', 'Primal', 'Groudon', 'Rayquaza', 'Rayquaza', 'Mega', 'Rayquaza', 'Jirachi', 'Deoxys', 'Normal', 'Forme', 'Deoxys', 'Attack', 'Forme', 'Deoxys', 'Defense', 'Forme', 'Deoxys', 'Speed', 'Forme', 'Uxie', 'Mesprit', 'Azelf', 'Dialga', 'Palkia', 'Heatran', 'Regigigas', 'Giratina', 'Altered', 'Forme', 'Giratina', 'Origin', 'Forme', 'Darkrai', 'Shaymin', 'Land', 'Forme', 'Shaymin', 'Sky', 'Forme', 'Arceus', 'Victini', 'Cobalion', 'Terrakion', 'Virizion', 'Tornadus', 'Incarnate', 'Forme', 'Tornadus', 'Therian', 'Forme', 'Thundurus', 'Incarnate', 'Forme', 'Thundurus', 'Therian', 'Forme', 'Reshiram', 'Zekrom', 'Landorus', 'Incarnate', 'Forme', 'Landorus', 'Therian', 'Form

In [ ]:
#전설의 포켓몬이 가장 많이 가지는 10개의 토큰 확인
from collections import Counter

most_common = Counter(token_set).most_common(10)

for token, _ in most_common:
    pokemon[f"{token}"] = pokemon["Name"].str.contains(token)

pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,...,Forme,Mega,Mewtwo,Kyurem,Deoxys,Hoopa,Latias,Latios,Kyogre,Groudon
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,...,False,False,False,False,False,False,False,False,False,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,...,False,False,False,False,False,False,False,False,False,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,...,False,False,False,False,False,False,False,False,False,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,...,False,True,False,False,False,False,False,False,False,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,...,False,False,False,False,False,False,False,False,False,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,...,False,False,False,False,False,False,False,False,False,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,...,False,False,False,False,False,False,False,False,False,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,...,False,True,False,False,False,False,False,False,False,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,...,False,True,False,False,False,False,False,False,False,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,...,False,False,False,False,False,False,False,False,False,False


#####정리하면, 전설의 포켓몬과 일반 포켓몬을 구분할 수 있는 유의미한 feature는,

1. Total 값의 크기 (클수록 전설의 포켓몬)
2. name의 길이 (길수록 전설의 포켓몬)
3. name의 특정 토큰 (가장 빈도가 높은 10가지 추출하여 컬럼으로 추가하였음)



---


##Step 2. 데이터 전처리

### 이름의 길이

이름의 길이는 전설의 포켓몬을 판별하는 중요한 특징이었기에, 관련된 별도의 컬럼 생성

In [ ]:
#포켓몬 데이터 전체에 이름의 글자수를 저장하는 name_count 컬럼 생성
pokemon["name_count"] = pokemon["Name"].apply(lambda i: len(i))

#이름의 길이가 10 이상이면 True, 미만이면 False를 가지는 long_name 컬럼을 생성
pokemon["long_name"] = pokemon["name_count"] >= 10

###Type을 원-핫 인코딩

모델에 문자형 데이터를 사용할 수 없으므로,   
숫자형으로 인코딩하되,   
정수형은 타입 간의 거리 관계가 있는 것으로 해석할 여지가 있으므로 원-핫 인코딩 수행

In [ ]:
types = list(set(pokemon["Type 1"]) | set(pokemon["Type 2"]))

for t in types:
    pokemon[t] = (pokemon["Type 1"] == t) | (pokemon["Type 2"] == t)

pokemon[[["Type 1", "Type 2"] + types][0]].head()

,Type 1,Type 2,Psychic,Fairy,Bug,Ground,Flying,Water,Fire,Ice,...,Rock,Fighting,Grass,NaN,Ghost,Normal,Dragon,Poison,Electric,Dark
0,Grass,Poison,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,False
1,Grass,Poison,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,False
2,Grass,Poison,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,False
3,Grass,Poison,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,False
4,Fire,NaN,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False




---

##Step 3. 기본 모델 학습

#### 학습 데이터 준비

In [ ]:
from sklearn.model_selection import train_test_split

#의미없거나 문자형인 데이터를 제외한 feature 지정
features = ['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation']
#타겟 지정
target = 'Legendary'

#각 데이터를 변수에 담기
basic_X = original_data[features]
basic_y = original_data[target]

#학습 데이터 분리
basic_X_train, basic_X_test, basic_y_train, basic_y_test = train_test_split(basic_X, basic_y, test_size=0.2, random_state=15)


####학습

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

basic_model = DecisionTreeClassifier(random_state=1)
basic_model.fit(basic_X_train, basic_y_train)
basic_y_pred = basic_model.predict(basic_X_test) #예측

print(classification_report(basic_y_test, basic_y_pred))

              precision    recall  f1-score   support

       False       0.97      0.98      0.98       147
        True       0.75      0.69      0.72        13

    accuracy                           0.96       160
   macro avg       0.86      0.84      0.85       160
weighted avg       0.95      0.96      0.96       160



---
## Step 4. 베이스라인 모델 학습 및 분석

####학습 데이터셋 준비

In [ ]:
#문자열 데이터를 갖는 컬럼과 target 컬럼을 제외한 feature 컬럼 생성
features = ['Total', 'HP', 'Attack', 'Defense','Sp. Atk', 'Sp. Def', 'Speed', 'Generation',
            'name_count','long_name', 'Forme', 'Mega', 'Mewtwo','Deoxys', 'Kyurem', 'Latias', 'Latios',
            'Kyogre', 'Groudon', 'Hoopa','Poison', 'Ground', 'Flying', 'Normal', 'Water', 'Fire',
            'Electric','Rock', 'Dark', 'Fairy', 'Steel', 'Ghost', 'Psychic', 'Ice', 'Bug', 'Grass', 'Dragon', 'Fighting']

#target 컬럼
target = 'Legendary'

#각 컬럼을 변수에 집어넣기
X = pokemon[features]
y = pokemon[target]

#학습 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(640, 38) (640,)
(160, 38) (160,)


---

####의사 결정 트리 모델 학습시키기


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

model = DecisionTreeClassifier(random_state=25) #모델 준비
model.fit(X_train, y_train) #모델 학습
y_pred = model.predict(X_test) #예측

print(classification_report(y_test, y_pred)) #예측 결과 확인

              precision    recall  f1-score   support

       False       0.99      0.96      0.98       147
        True       0.67      0.92      0.77        13

    accuracy                           0.96       160
   macro avg       0.83      0.94      0.87       160
weighted avg       0.97      0.96      0.96       160



####해석
1. 일반 포켓몬 예측은 우수한 성능을 보이나, 전설 포켓몬 예측은 상대적으로 낮은 성능을 가짐
  * 데이터 불균형이 심각한 것에 따른 결과로 보임.
  * 불균형에 덜 민감한 모델로 개선할 필요 있음

2. 전설 포켓몬의 precision 부문에서 낮은 성능(0.67)을 보임.   
  * 일반 포켓몬을 전설로 잘못 예측하는 경우가 많음을 의미

- 결론: 데이터 불균형에 덜 민감한 random forest로 재학습 시도해보자.



---

## Step 5. 비교 모델 학습 및 분석

####random forest 모델 학습 및 분석

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=1) #모델 준비
rf_model.fit(X_train, y_train) #모델 학습

# 예측 및 평가
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.97      0.98      0.98       147
        True       0.75      0.69      0.72        13

    accuracy                           0.96       160
   macro avg       0.86      0.84      0.85       160
weighted avg       0.95      0.96      0.96       160



1. precision 의 큰 개선이 있었음 (0.67 -> 0.83)
  * 일반 포켓몬이 전설로 잘못 예측되는 경우가 줄었음
  * 게임 내에서, 전설 포켓몬이 출현할 경우 특수한 스킨이 발현되는 등의 이펙트가 있음. precision 이 상승함으로써 불필요한 리소스가 발생할 확률을 줄이는 데에 긍정적인 역할을 할 것임

2. recall의 큰 감소가 있었음 (0.92 -> 0.77)
  * 전설 포켓몬이 일반 포켓몬으로 예측되는 경우가 많아졌음

- 분석
  * 이번 문제는 전설 포켓몬을 찾아야 하는 것에 초점이 맞춰져 있기 때문에, recall이 precision 대비 더 중요함.
  * 따라서 이전 모델 대비 성능이 낮아졌다고 판단할 수 있음.



---



####random forest 모델 개선

recall 값을 높이기 위해,    
전설 포켓몬 클래스에 더 많은 가중치 부여를 시도하기로 함.

모델 생성 시, class_weight="balance" 옵션을 추가하여 클래스 비율에 따른 가중치 설정

In [ ]:
rf_model = RandomForestClassifier(random_state=1, class_weight="balanced") #가중치 조정한 모델 준비
rf_model.fit(X_train, y_train) #모델 학습

# 예측 및 평가
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.98      0.97      0.97       147
        True       0.67      0.77      0.71        13

    accuracy                           0.95       160
   macro avg       0.82      0.87      0.84       160
weighted avg       0.95      0.95      0.95       160



- 분석:
  * recall 값은 올랐으나, 전반적인 성능(f1-score)는 세 모델 대비 가장 낮음.
  * 가중치 외의 하이퍼파라미터를 조정해 볼 필요가 있음.




---



#### Grid Search 모델 학습 및 분석

여러가지 하이퍼파라미터에 대해 한꺼번에 조정하면서도
최적의 하이퍼파라미터 조합을 찾아낼 수 있도록
grid search 모델 사용

In [ ]:
from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터 후보
param_grid = {
    "n_estimators": [100, 150, 200, 250, 300],
    "max_depth": [5, 10, 15, 20, 25, 30],
    "min_samples_split": [2, 5, 10, 15, 20],
    "class_weight": ["balanced"]
}

# Grid Search 설정
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=1),
    param_grid=param_grid,
    scoring="recall",  # Recall 기준으로 최적화
    cv=5,  # 5-Fold Cross Validation
    n_jobs=-1  # 병렬 처리
)

# Grid Search 실행
grid_search.fit(X_train, y_train)

# 최적의 파라미터 출력
print("Best Parameters:", grid_search.best_params_)

# 최적 모델로 예측
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))

Best Parameters: {'class_weight': 'balanced', 'max_depth': 5, 'min_samples_split': 15, 'n_estimators': 200}
              precision    recall  f1-score   support

       False       1.00      0.93      0.96       147
        True       0.57      1.00      0.72        13

    accuracy                           0.94       160
   macro avg       0.78      0.97      0.84       160
weighted avg       0.96      0.94      0.95       160



- 분석:
  * scoring을 recall 기준으로 둔 결과,   
recall 값이 1이 되는 놀라운 결과가 나왔다.

  * 하지만 precision 은 현격히 저하되어,  
f1-score 값은 이전 모델들과 큰 차이를 보이지 않게 되었다.



---


##Step 6. 결론 및 회고

##프로젝트 평가
- 기본 모델이 동작하는 코드가 제출되었는가?
  * Step 3.로 기본 모델을 만들었으며, 성공적으로 실행되었다.

- 데이터 전처리
  * 성공적으로 데이터 전처리가 수행되었으며, 각각의 설명이 과정이 포함되었다.

- 베이스라인 모델 학습 성공 및 분석
  * Step 4. 로 전처리 된 데이터를 기반으로 한 모델 학습 및 실행이 수행되었으며, 이에 대한 분석이 이루어졌다.

- 비교 모델 학습 성공 및 분석
  * 베이스라인 모델에서의 결과를 바탕으로, 추가적인 모델 두가지를 실험해보았다.
    * random_forest
    * grid search

</br>

##회고
* 이전 모델에 대한 보완을 연쇄적으로 시도하였으나, 전반적으로 눈에 띄는 개선을 보인 시도가 없었다.
  * 베이스라인 모델의 결과로 데이터 불균형에 덜 민감한 random forest 모델을 사용하였으나, recall 값 저하됨.
  * random forest 개선을 위해 class_weight 를 주었고, recall은 개선되었으나 precision 저하
  * 아예 다양한 하이퍼 파라미터를 한꺼번에 조정하여 전반적 성능을 개선하고자 grid_search 모델을 사용하여, recall을 1까지 맞추는 결과를 얻어내었으나, precision이 역대 최저로 떨어짐.